In [1]:
def convert_ply_float_to_int(input_path, output_path, attr="object_id"):
    with open(input_path, "r") as f:
        lines = f.readlines()

    header = []
    body = []
    in_header = True
    attr_line_index = -1

    for i, line in enumerate(lines):
        if in_header:
            header.append(line)
            if line.startswith("property float " + attr):
                attr_line_index = i
                header[-1] = line.replace("property float ", "property int ")
            if line.strip() == "end_header":
                in_header = False
        else:
            body.append(line)

    # Rewrite body (convert the attribute column float->int)
    new_body = []
    for line in body:
        parts = line.split()
        if len(parts) > 3:  # vertex lines
            # Object ID is last column
            parts[-1] = str(int(float(parts[-1])))
        new_body.append(" ".join(parts) + "\n")

    with open(output_path, "w") as f:
        f.writelines(header + new_body)

    print("Wrote:", output_path)

In [2]:
# input = "/home/student/ss/data/scene_datasets/trial/try/cube_attributes_semantic.ply"
# output = "/home/student/ss/data/scene_datasets/trial/try/cube_attributes_semantic.ply"

# convert_ply_float_to_int(input, output, attr="object_id")

In [3]:
import struct

def convert_ply_float_to_int_binary(input_path, output_path, attr="object_id"):
    with open(input_path, "rb") as f:
        content = f.read()

    # Find the end of the header
    header_end = content.find(b"end_header\n") + len(b"end_header\n")
    if header_end == -1:
        raise RuntimeError("PLY header not found")

    # Decode header as ASCII
    header = content[:header_end].decode("ascii")

    # Replace float property with int
    new_header = header.replace(f"property float {attr}", f"property int {attr}")

    # Parse number of vertices
    vertex_count = 0
    for line in header.split("\n"):
        if line.startswith("element vertex"):
            vertex_count = int(line.split()[2])

    # Extract property list for vertices
    lines = header.split("\n")
    vertex_props = []
    reading = False
    for line in lines:
        if line.startswith("element vertex"):
            reading = True
            continue
        if reading:
            if line.startswith("property"):
                vertex_props.append(line)
            elif line.startswith("element"):
                break

    prop_count = len(vertex_props)

    # Which property index is object_id?
    object_id_index = [
        i for i, p in enumerate(vertex_props)
        if p.endswith(f" {attr}")
    ][0]

    # Vertex block is prop_count * 4 bytes per vertex
    vertex_data_size = vertex_count * prop_count * 4
    vertex_data = content[header_end: header_end + vertex_data_size]

    # Unpack all float32 values
    floats = list(struct.unpack("<" + "f" * (prop_count * vertex_count), vertex_data))

    # Convert object_id from float → int
    for v in range(vertex_count):
        idx = v * prop_count + object_id_index
        floats[idx] = int(floats[idx])

    # Repack: for object_id use int32, others remain float32
    new_vertex_data = bytearray()
    for v in range(vertex_count):
        for i in range(prop_count):
            val = floats[v * prop_count + i]
            if i == object_id_index:
                new_vertex_data += struct.pack("<i", int(val))
            else:
                new_vertex_data += struct.pack("<f", float(val))

    # Rest of file (faces)
    remainder = content[header_end + vertex_data_size:]

    # Write new PLY
    with open(output_path, "wb") as f:
        f.write(new_header.encode("ascii"))
        f.write(new_vertex_data)
        f.write(remainder)

    print("Converted binary PLY written to:", output_path)


In [4]:
input = "/home/student/ss/data/scene_datasets/sim/stages/cube_stage_semantic_objectid.ply"
output = "/home/student/ss/data/scene_datasets/sim/stages/cube_stage_semantic_objectidint.ply"

convert_ply_float_to_int_binary(input, output, attr="object_id")

error: 'i' format requires -2147483648 <= number <= 2147483647

In [ ]:
from plyfile import PlyData, PlyElement
import numpy as np
import struct

input_file = "/home/student/ss/data/scene_datasets/sim/stages/cube_stage_semantic_objectid.ply"
output_file = "/home/student/ss/data/scene_datasets/sim/stages/cube_stage_semantic_objectidint.ply"

# Load the binary PLY using plyfile
plydata = PlyData.read(input_file)

# Extract vertex data
vertices = plydata['vertex'].data

# Create new structured array with int object_id
new_vertex_dtype = [('x','f4'), ('y','f4'), ('z','f4'),
                    ('red','u1'), ('green','u1'), ('blue','u1'),
                    ('alpha','u1'),
                    ('object_id','i4')]

new_vertices = np.empty(len(vertices), dtype=new_vertex_dtype)

# Copy data and convert object_id to int
new_vertices['x'] = vertices['x']
new_vertices['y'] = vertices['y']
new_vertices['z'] = vertices['z']
new_vertices['red'] = vertices['red']
new_vertices['green'] = vertices['green']
new_vertices['blue'] = vertices['blue']
new_vertices['alpha'] = vertices['alpha']
# new_vertices['s'] = vertices['s']
# new_vertices['t'] = vertices['t']
new_vertices['object_id'] = vertices['object_id'].astype(np.int32)

# Write fixed binary PLY
ply_el = PlyElement.describe(new_vertices, 'vertex')
PlyData([ply_el, plydata['face']], text=False).write(output_file)

print("Fixed PLY written to:", output_file)

print(len(plydata['vertex'].data))



Fixed PLY written to: /home/student/ss/data/scene_datasets/sim/stages/cube_stage_semantic_objectidint.ply
8


In [ ]:
from plyfile import PlyData, PlyElement
import numpy as np
import os

# Input Blender-exported PLY
input_file = "/home/student/ss/data/scene_datasets/trial/try/cube_attributes_semantic.ply"
# Output SoundSpaces-ready PLY
output_file = "/home/student/ss/data/scene_datasets/trial/try/cube_attributes_semantic.ply"

# Check if input exists
if not os.path.isfile(input_file):
    raise FileNotFoundError(f"Input file not found: {input_file}")

# Load binary PLY
plydata = PlyData.read(input_file)

# Extract vertex and face data
vertices = plydata['vertex'].data
faces = plydata['face'].data

num_vertices = len(vertices)
num_faces = len(faces)

# Define new structured array dtype: x,y,z, RGB, object_id (object_id last)
vertex_dtype = [
    ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
    ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'),
    ('object_id', 'i4')
]

new_vertices = np.empty(num_vertices, dtype=vertex_dtype)

# Copy positions
new_vertices['x'] = vertices['x']
new_vertices['y'] = vertices['y']
new_vertices['z'] = vertices['z']

# Copy or create vertex colors
if all(c in vertices.dtype.names for c in ['red', 'green', 'blue']):
    new_vertices['red'] = vertices['red']
    new_vertices['green'] = vertices['green']
    new_vertices['blue'] = vertices['blue']
else:
    # Assign default white color if missing
    new_vertices['red'] = 255
    new_vertices['green'] = 255
    new_vertices['blue'] = 255

# Convert object_id to int32
if 'object_id' in vertices.dtype.names:
    new_vertices['object_id'] = vertices['object_id'].astype(np.int32)
else:
    # Default to 0 if missing
    new_vertices['object_id'] = np.zeros(num_vertices, dtype=np.int32)

# Create PLY elements
vertex_element = PlyElement.describe(new_vertices, 'vertex')
face_element = PlyElement.describe(faces, 'face')

# Write binary little-endian PLY
PlyData([vertex_element, face_element], text=False).write(output_file)

print(f"✅ SoundSpaces-ready PLY written to: {output_file}")
print(f"Number of vertices: {num_vertices}, Number of faces: {num_faces}")


✅ SoundSpaces-ready PLY written to: /home/student/ss/data/scene_datasets/trial/try/cube_attributes_semantic_FIXED.ply
Number of vertices: 14, Number of faces: 12


In [ ]:
from plyfile import PlyData

ply = PlyData.read(input_file)

# Get vertex data as structured array
vertex_data = ply['vertex'].data
face_data = ply['face'].data

# Print vertex property names
print("Vertex properties:", vertex_data.dtype.names)

# Print number of vertices and faces
print("Number of vertices:", len(vertex_data))
print("Number of faces:", len(face_data))

print(vertex_data.dtype)
print(face_data.dtype)
face_indices = face_data['vertex_indices']
max_index = max(max(f) for f in face_indices)
print("Max face index:", max_index)


Vertex properties: ('x', 'y', 'z', 'red', 'green', 'blue', 'object_id')
Number of vertices: 14
Number of faces: 12
[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'), ('object_id', '<i4')]
[('vertex_indices', 'O')]
Max face index: 13


In [ ]:
import json
import numpy as np
from plyfile import PlyData, PlyElement

# ===== INPUT/OUTPUT FILES =====
ply_file = "/home/student/ss/data/scene_datasets/trial/try/cube_attributes_semantic.ply"
material_json_file = "/home/student/ss/data/mp3d_material_config.json"
output_file = "/home/student/ss/data/scene_datasets/trial/try/cube_attributes_semantic.ply"

# ===== LOAD PLY =====
plydata = PlyData.read(ply_file)
vertex_data = plydata['vertex'].data
face_data = plydata['face'].data
num_vertices = len(vertex_data)
num_faces = len(face_data)

print(f"Number of vertices: {num_vertices}, Number of faces: {num_faces}")

# ===== VALIDATE FACE INDICES =====
print("\nChecking face indices...")
faces_ok = True
for i, f in enumerate(face_data['vertex_indices']):
    for idx in f:
        if idx >= num_vertices or idx < 0:
            print(f"Face {i} has out-of-range index: {f}")
            faces_ok = False

if faces_ok:
    print("All face indices are valid ✅")

# ===== VALIDATE AND FIX object_id VS MATERIAL JSON =====
print("\nChecking object_ids against material JSON...")

with open(material_json_file, 'r') as f:
    material_config = json.load(f)

material_list = material_config["materials"]
num_materials = len(material_list)  # valid object_id indices: 0 .. num_materials-1

# Copy vertex data to a structured array we can modify
vertex_dtype = vertex_data.dtype.descr
new_vertices = np.empty(len(vertex_data), dtype=vertex_dtype)
for name in vertex_data.dtype.names:
    new_vertices[name] = vertex_data[name]

# Check and fix object_id
object_ids = np.unique(new_vertices['object_id'])
missing_ids = []

for i in range(len(new_vertices)):
    oid = new_vertices['object_id'][i]
    if oid >= num_materials or oid < 0:
        missing_ids.append(oid)
        # Map invalid ID to default material (0)
        new_vertices['object_id'][i] = 0

if missing_ids:
    print("Warning ⚠: Fixed invalid object_ids:", set(missing_ids))
else:
    print("All object_ids valid ✅")

# ===== WRITE FIXED PLY =====
# Ensure object_id is the last property before end_header
# PlyElement.describe preserves the order of the dtype
vertex_element = PlyElement.describe(new_vertices, 'vertex')
face_element = PlyElement.describe(face_data, 'face')
PlyData([vertex_element, face_element], text=False).write(output_file)

print("\n✅ Fixed SoundSpaces-ready PLY written to:", output_file)

Number of vertices: 14, Number of faces: 12

Checking face indices...
All face indices are valid ✅

Checking object_ids against material JSON...
All object_ids valid ✅

✅ Fixed SoundSpaces-ready PLY written to: /home/student/ss/data/scene_datasets/trial/try/cube_attributes_semantic.ply


In [ ]:
import numpy as np
from plyfile import PlyData

plydata = PlyData.read(output_file)
vertices = np.vstack([plydata['vertex']['x'],
                      plydata['vertex']['y'],
                      plydata['vertex']['z']]).T
faces = plydata['face']['vertex_indices']

# Flatten all face indices
all_indices = np.hstack(faces)
print("Max vertex index:", all_indices.max())
print("Min vertex index:", all_indices.min())

# Check for degenerate faces (rank < 3)
degenerate = [i for i, f in enumerate(faces) 
              if np.linalg.matrix_rank(vertices[f]) < 3]
print("Degenerate faces:", degenerate)


Max vertex index: 13
Min vertex index: 0
Degenerate faces: []


In [ ]:
import json
import numpy as np
from plyfile import PlyData

# ===== INPUT FILES =====
ply_file = "/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic_from_blender.ply"
material_json_file = "/home/student/ss/data/try_material_config.json"

# ===== LOAD PLY =====
plydata = PlyData.read(ply_file)
vertex_data = plydata['vertex'].data
face_data = plydata['face'].data
num_vertices = len(vertex_data)
num_faces = len(face_data)

print(f"Vertices: {num_vertices}, Faces: {num_faces}")

# ===== CHECK FOR NaNs / INFs =====
coords = np.vstack([vertex_data['x'], vertex_data['y'], vertex_data['z']]).T
nan_mask = np.isnan(coords).any(axis=1)
inf_mask = np.isinf(coords).any(axis=1)

if nan_mask.any():
    print("Warning ⚠: vertices with NaN coordinates at indices:", np.where(nan_mask)[0])
if inf_mask.any():
    print("Warning ⚠: vertices with infinite coordinates at indices:", np.where(inf_mask)[0])
if not nan_mask.any() and not inf_mask.any():
    print("No NaNs or Infs in vertex coordinates ✅")

# ===== CHECK FOR DUPLICATE VERTICES =====
coords_tuple = [tuple(v) for v in coords]
duplicates = [i for i, v in enumerate(coords_tuple) if coords_tuple.count(v) > 1]
if duplicates:
    print("Warning ⚠: Duplicate vertices found at indices:", duplicates)
else:
    print("No duplicate vertices ✅")

# ===== CHECK FACE INDICES =====
faces = face_data['vertex_indices']
all_indices = np.hstack(faces)
if all_indices.max() >= num_vertices or all_indices.min() < 0:
    print("Warning ⚠: Face indices out of vertex range")
else:
    print("All face indices valid ✅")

# ===== CHECK FOR DEGENERATE FACES =====
degenerate_faces = [i for i, f in enumerate(faces)
                    if np.linalg.matrix_rank(coords[f]) < 3]
if degenerate_faces:
    print("Warning ⚠: Degenerate faces (rank < 3) at indices:", degenerate_faces)
else:
    print("No degenerate faces ✅")

# ===== VALIDATE object_id VS MATERIAL JSON =====
with open(material_json_file, 'r') as f:
    material_config = json.load(f)

materials = material_config["materials"]
num_materials = len(materials)
object_ids = np.unique(vertex_data['object_id'])

invalid_ids = [oid for oid in object_ids if oid < 0 or oid >= num_materials]
if invalid_ids:
    print(f"Warning ⚠: object_ids not in material JSON: {invalid_ids}")
else:
    print("All object_ids valid ✅")

# ===== SUMMARY =====
print("\nPLY Validation Complete!")
print(f"Vertices: {num_vertices}, Faces: {num_faces}")
print(f"Unique object_ids: {object_ids}")


Vertices: 8, Faces: 12
No NaNs or Infs in vertex coordinates ✅
No duplicate vertices ✅
All face indices valid ✅
No degenerate faces ✅
All object_ids valid ✅

PLY Validation Complete!
Vertices: 8, Faces: 12
Unique object_ids: [0 1]


In [ ]:
import numpy as np
from plyfile import PlyData, PlyElement

ply_file = "/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic_from_blender.ply"
output_file = "/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic.ply"

plydata = PlyData.read(ply_file)
vertices = np.array([(v['x'], v['y'], v['z'], v['red'], v['green'], v['blue'], v['object_id'])
                     for v in plydata['vertex'].data],
                    dtype=[('x','f4'), ('y','f4'), ('z','f4'),
                           ('red','u1'),('green','u1'),('blue','u1'),('object_id','i4')])

faces = plydata['face']['vertex_indices']

# Remove duplicate vertices
coords = np.vstack([vertices['x'], vertices['y'], vertices['z']]).T
_, unique_indices, inverse_map = np.unique(coords, axis=0, return_index=True, return_inverse=True)
new_vertices = vertices[unique_indices]

# Update face indices
new_faces = np.array([inverse_map[f] for f in faces], dtype=object)

# Create new PLY elements
vertex_element = PlyElement.describe(new_vertices, 'vertex')
face_element = PlyElement.describe(np.array([(f,) for f in new_faces], dtype=[('vertex_indices','O')]), 'face')

# Write fixed PLY
PlyData([vertex_element, face_element], text=False).write(output_file)
print("Fixed PLY written to:", output_file)

Fixed PLY written to: /home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic.ply


In [ ]:
from plyfile import PlyData

ply = PlyData.read(output_file)
vertices = ply['vertex'].data
print(len(vertices), len(vertices['object_id']))

faces = ply['face'].data['vertex_indices']
for f in faces:
    for idx in f:
        if idx >= len(vertices):
            print("Invalid index!", idx)

8 8


In [ ]:
import trimesh
import numpy as np

# Load GLB
glb = trimesh.load("/home/student/ss/data/scene_datasets/trial/try/cube_15153.glb")

# If GLB is a Scene, collect all vertices
if isinstance(glb, trimesh.Scene):
    glb_vertices = []
    for name, geom in glb.geometry.items():
        print("Found mesh:", name)
        glb_vertices.append(geom.vertices)
    glb_vertices = np.vstack(glb_vertices)
else:
    glb_vertices = glb.vertices

# Load PLY
ply = trimesh.load("/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic.ply")
ply_vertices = ply.vertices

print("GLB verts:", len(glb_vertices))
print("PLY verts:", len(ply_vertices))

# Check if vertex counts match
if len(glb_vertices) != len(ply_vertices):
    print("⚠️ Vertex count mismatch! AudioSensor will likely crash.")
else:
    # Compare positions
    diff = glb_vertices - ply_vertices
    print("Max difference per axis:", np.abs(diff).max(axis=0))


Found mesh: Cube.001
GLB verts: 8
PLY verts: 8
Max difference per axis: [0. 0. 0.]


In [ ]:
import numpy as np
import trimesh
from scipy.spatial import cKDTree

# Load GLB
glb = trimesh.load("/home/student/ss/data/scene_datasets/trial/try/cube_15153.glb")
if isinstance(glb, trimesh.Scene):
    glb_vertices = np.vstack([geom.vertices for geom in glb.geometry.values()])
else:
    glb_vertices = glb.vertices

# Load PLY
ply = trimesh.load("/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic.ply")
ply_vertices = ply.vertices

# Build KD-tree from PLY vertices
tree = cKDTree(ply_vertices)

# For each GLB vertex, find nearest PLY vertex
_, idx = tree.query(glb_vertices, k=1)

# Reorder PLY vertices and colors/object_ids
ply_vertices_reordered = ply_vertices[idx]
ply_colors_reordered = ply.visual.vertex_colors[idx]

# Update the PLY mesh
ply.vertices = ply_vertices_reordered
ply.visual.vertex_colors = ply_colors_reordered

# Save fixed PLY
ply.export("/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic_fixed.ply")
print("PLY reordered to match GLB vertex order!")


PLY reordered to match GLB vertex order!


In [ ]:
print(ply.vertices)

print(glb_vertices)

assert ply.vertices.all() == glb_vertices.all()

[[-7.5 -7.5  7.5]
 [-7.5  7.5  7.5]
 [-7.5 -7.5 -7.5]
 [-7.5  7.5 -7.5]
 [ 7.5 -7.5  7.5]
 [ 7.5  7.5  7.5]
 [ 7.5 -7.5 -7.5]
 [ 7.5  7.5 -7.5]]
[[-7.5 -7.5  7.5]
 [-7.5  7.5  7.5]
 [-7.5 -7.5 -7.5]
 [-7.5  7.5 -7.5]
 [ 7.5 -7.5  7.5]
 [ 7.5  7.5  7.5]
 [ 7.5 -7.5 -7.5]
 [ 7.5  7.5 -7.5]]


In [ ]:
import numpy as np
import trimesh

ply = trimesh.load("/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic_fixed.ply")
print("Vertices:", ply.vertices.shape)
print("Faces:", ply.faces.shape)
print("Any NaNs:", np.isnan(ply.vertices).any())


Vertices: (8, 3)
Faces: (12, 3)
Any NaNs: False


In [ ]:
print(ply)

<trimesh.Trimesh(vertices.shape=(8, 3), faces.shape=(12, 3))>


In [ ]:
import numpy as np
import trimesh
from scipy.spatial import cKDTree
from plyfile import PlyData, PlyElement

# Paths
glb_path = "/home/student/ss/data/scene_datasets/trial/try/cube_15153.glb"
ply_path = "/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic.ply"
output_path = "/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic_fixed.ply"

# Load GLB
glb = trimesh.load(glb_path)
if isinstance(glb, trimesh.Scene):
    glb_vertices = np.vstack([geom.vertices for geom in glb.geometry.values()])
else:
    glb_vertices = glb.vertices

# Load PLY with plyfile to access object_id
plydata = PlyData.read(ply_path)
vertex_data = plydata['vertex'].data
ply_vertices = np.vstack([vertex_data['x'], vertex_data['y'], vertex_data['z']]).T
ply_colors = np.vstack([vertex_data['red'], vertex_data['green'], vertex_data['blue']]).T
obj_ids = vertex_data['object_id']

# Build KD-tree from PLY vertices
tree = cKDTree(ply_vertices)

# For each GLB vertex, find nearest PLY vertex
_, idx = tree.query(glb_vertices, k=1)

# Reorder PLY vertices, colors, and object_ids
ply_vertices_reordered = ply_vertices[idx]
ply_colors_reordered = ply_colors[idx]
obj_ids_reordered = obj_ids[idx]

# Create structured array for export
vertex_dtype = [
    ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
    ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'),
    ('object_id', 'i4')
]
vertex_data_out = np.empty(len(ply_vertices_reordered), dtype=vertex_dtype)
vertex_data_out['x'] = ply_vertices_reordered[:, 0]
vertex_data_out['y'] = ply_vertices_reordered[:, 1]
vertex_data_out['z'] = ply_vertices_reordered[:, 2]
vertex_data_out['red'] = ply_colors_reordered[:, 0]
vertex_data_out['green'] = ply_colors_reordered[:, 1]
vertex_data_out['blue'] = ply_colors_reordered[:, 2]
vertex_data_out['object_id'] = obj_ids_reordered

# Faces (unchanged)
faces = np.array(plydata['face'].data, dtype=[('vertex_indices', 'i4', (3,))])

# Export PLY
ply_element = PlyElement.describe(vertex_data_out, 'vertex')
face_element = PlyElement.describe(faces, 'face')
PlyData([ply_element, face_element], text=False).write(output_path)

print(f"PLY reordered and object_id preserved! Saved to: {output_path}")


PLY reordered and object_id preserved! Saved to: /home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic_fixed.ply


In [ ]:
import numpy as np
import trimesh

def validate_semantic_ply(ply_file, expected_vertices=None):
    """
    Validates a Habitat semantic PLY mesh for audio simulation.

    Parameters
    ----------
    ply_file : str
        Path to the PLY file.
    expected_vertices : int or None
        If provided, checks that the number of vertices matches.

    Returns
    -------
    bool
        True if the PLY passes all checks, False otherwise.
    """

    mesh = trimesh.load(ply_file)
    
    # --- Extract vertices and faces ---
    if isinstance(mesh, trimesh.Scene):
        # Combine all geometries
        verts_list = []
        faces_list = []
        for name, geom in mesh.geometry.items():
            verts_list.append(geom.vertices)
            faces_list.append(geom.faces)
        vertices = np.vstack(verts_list)
        faces = np.vstack(faces_list)
    else:
        vertices = mesh.vertices
        faces = mesh.faces
    
    print(f"Vertices: {vertices.shape[0]}, Faces: {faces.shape[0]}")
    
    # --- Vertex validation ---
    if np.isnan(vertices).any():
        print("ERROR: NaNs in vertex coordinates!")
        return False
    if np.isinf(vertices).any():
        print("ERROR: Infs in vertex coordinates!")
        return False
    
    if expected_vertices is not None and vertices.shape[0] != expected_vertices:
        print(f"WARNING: Vertex count {vertices.shape[0]} != expected {expected_vertices}")

    # --- Face validation ---
    if faces.max() >= vertices.shape[0] or faces.min() < 0:
        print("ERROR: Face indices out of range!")
        return False
    degenerate_faces = [i for i, f in enumerate(faces) if len(set(f)) < 3]
    if degenerate_faces:
        print(f"ERROR: Degenerate faces at indices {degenerate_faces}")
        return False
    
    # --- Object ID validation ---
    obj_ids = None

    # --- Attempt via trimesh ---
    if hasattr(mesh, "metadata") and "ply_raw" in mesh.metadata:
        # If object_id is in structured array
        vertex_data = mesh.metadata["ply_raw"]["vertex"]
        obj_ids = vertex_data.get("object_id", None)

    if obj_ids is None:
        # Try accessing vertex attributes via trimesh
        try:
            obj_ids = mesh.vertex_attributes["object_id"]
        except (AttributeError, KeyError):
            obj_ids = None

    # --- Fallback to plyfile if needed ---
    if obj_ids is None and ply_path is not None:
        try:
            plydata = PlyData.read(ply_path)
            obj_ids = np.array(plydata['vertex'].data['object_id'])
        except Exception as e:
            print(f"WARNING: Failed to read object_id from PLY: {e}")
            obj_ids = None

    # --- Validate object_ids ---
    if obj_ids is not None:
        obj_ids = np.array(obj_ids)
        print("Object_id length: ", len(obj_ids))
        print("The object ids: ", object_ids)
        print("Object_id dtype: ", object_ids.dtype)
        if not np.issubdtype(obj_ids.dtype, np.integer):
            print(f"ERROR: object_id array is not integer, dtype={obj_ids.dtype}")
            return False
        if obj_ids.min() < 0:
            print("ERROR: object_id has negative values!")
            return False
        print(f"Unique object_ids: {np.unique(obj_ids)}")
    else:
        print("WARNING: object_id attribute not found. Habitat will fail.")
        return False

    print("✅ PLY semantic mesh validation passed!")
    return True

# --- Example usage ---
ply_file = "/home/student/ss/data/scene_datasets/trial/try/cube_15153_semantic.ply"
validate_semantic_ply(ply_file, expected_vertices=8)


Vertices: 8, Faces: 12
Object_id length:  8
The object ids:  [0 1]
Object_id dtype:  int32
Unique object_ids: [0 1]
✅ PLY semantic mesh validation passed!


True

In [ ]:
from plyfile import PlyData, PlyElement
import numpy as np

# bl = PlyData.read("/home/student/ss/data/scene_datasets/sim/stages/treated_room_cube.ply")
# bl = PlyData.read("/home/student/ss/data/scene_datasets/sim/stages/treated_room.ply")
bl = PlyData.read("/home/student/ss/data/scene_datasets/sim/stages/treated_room_object_semantic.ply")
# bl = PlyData.read("/home/student/ss/data/scene_datasets/sim/objects/treated_room_object_bbox.ply")
# bl = PlyData.read("/home/student/ss/data/scene_datasets/sim/stages/room_stage_semantic.ply")
# bl = PlyData.read("/home/student/ss/data/scene_datasets/mp3d/17DRP5sb8fy/17DRP5sb8fy_semantic_noObjectID.ply")
# bl = PlyData.read("/home/student/ss/data/scene_datasets/sim/stages/cube_stage_semantic.ply")

# Get blender vertices
verts = np.array(bl['vertex'].data)

# build new vertex dtype
dtype = [
    ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
    ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'),
    ('object_id', 'i4')
]

new_verts = np.empty(len(verts), dtype=dtype)
new_verts['x'] = verts['x']
new_verts['y'] = verts['y']
new_verts['z'] = verts['z']
new_verts['red'] = verts['red']
new_verts['green'] = verts['green']
new_verts['blue'] = verts['blue']
for v in range(len(new_verts)):
    if new_verts['red'][v] == 254:
        new_verts['object_id'][v] = 1
    elif new_verts['green'][v] == 254:
        new_verts['object_id'][v] = 0
    elif new_verts['blue'][v] == 254:
        new_verts['object_id'][v] = 2
    else:
        new_verts['object_id'][v] = 3

# faces unchanged
faces = bl['face']

# # ASCII
ply = PlyData(
    [PlyElement.describe(new_verts, 'vertex'),
     faces],
    text=True)

ply.write("/home/student/ss/data/scene_datasets/sim/stages/treated_room_object_semantic_ASCII.ply")
# ply.write("/home/student/ss/data/scene_datasets/sim/stages/treated_room_cube_semantic_ASCII.ply")
# ply.write("/home/student/ss/data/scene_datasets/sim/objects/treated_room_object_bbox_semantic_ASCII.ply")
# ply.write("/home/student/ss/data/scene_datasets/sim/stages/cube_stage_semantic_ASCII.ply")

# BINARY
ply = PlyData(
    [
        PlyElement.describe(new_verts, 'vertex'),
        faces,
    ],
    byte_order='<')
# # ply.write("/home/student/ss/data/scene_datasets/sim/stages/treated_room_with_speaker_and_mic_semantic.ply")
# ply.write("/home/student/ss/data/scene_datasets/sim/stages/treated_room_cube_semantic.ply")
ply.write("/home/student/ss/data/scene_datasets/sim/stages/treated_room_object_semantic.ply")
# ply.write("/home/student/ss/data/scene_datasets/sim/objects/treated_room_object_bbox_semantic.ply")
# # ply.write("/home/student/ss/data/scene_datasets/sim/stages/treated_room_semantic.ply")
# ply.write("/home/student/ss/data/scene_datasets/sim/stages/cube_stage_semantic.ply")


FileNotFoundError: [Errno 2] No such file or directory: '/home/student/ss/data/scene_datasets/sim/stages/treated_room_object.ply'

In [35]:
print(len(new_verts))
print(new_verts.dtype)
print(new_verts[-8:])
new_verts[-8:]['x'] += 1.0
new_verts[-8:]['y'] -= 1.0
print(new_verts[-8:])

ply = PlyData(
    [
        PlyElement.describe(new_verts, 'vertex'),
        faces,
    ],
    byte_order='<')

ply.write("/home/student/ss/data/scene_datasets/sim/stages/treated_room_object_moved_semantic.ply")

41
[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'), ('object_id', '<i4')]
[(2.5, -3.5, 0., 0, 0, 0, 3) (2.5, -3.5, 2., 0, 0, 0, 3)
 (2.5, -2.5, 0., 0, 0, 0, 3) (2.5, -2.5, 2., 0, 0, 0, 3)
 (3.5, -3.5, 0., 0, 0, 0, 3) (3.5, -3.5, 2., 0, 0, 0, 3)
 (3.5, -2.5, 0., 0, 0, 0, 3) (3.5, -2.5, 2., 0, 0, 0, 3)]
[(3.5, -4.5, 0., 0, 0, 0, 3) (3.5, -4.5, 2., 0, 0, 0, 3)
 (3.5, -3.5, 0., 0, 0, 0, 3) (3.5, -3.5, 2., 0, 0, 0, 3)
 (4.5, -4.5, 0., 0, 0, 0, 3) (4.5, -4.5, 2., 0, 0, 0, 3)
 (4.5, -3.5, 0., 0, 0, 0, 3) (4.5, -3.5, 2., 0, 0, 0, 3)]
